In [1]:
import pandas as pd
import numpy as np
import json
import sqlite3 as lite
from pathlib import Path
import os

In [2]:
pd.set_option("display.max_columns", 0)

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Shopee restaurant data

In [12]:
!ls drive/MyDrive/Data/'1 - raw data'/summary/

foody_columns_highlights.xlsx  hcmc_shopee_order_like.xlsx     shopee_hcm.xlsx
foody_hcm.xlsx		       shopee_columns_highlights.xlsx


In [13]:
shopee_df=pd.read_excel("drive/MyDrive/Data/1 - raw data/summary/hcmc_shopee_order_like.xlsx")

In [14]:
shopee_df

,Unnamed: 0,browsing_id,location_url,is_quality_merchant,cuisines,address,restaurant_name,restaurant_url,lat,lon,district_id,restaurant_id,has_phone,restaurant_short_description,brand_id,brand_url,brand_name,restaurant_count,limit_distance,categories,is_favorite,delivery_id,total_order_x,rating_total_review,rating_avg,rating_display_total_review,rating_app_link,is_subscribe,asap_is_available,is_city_alert,contract_type,delivery_fees,vat,confirm_language,service_type,user_favorite_count,delivery_week_days,delivery_service_by,delivery_service_fee,delivery_merchant_limit_distance,delivery_payment_methods,delivery_has_contract,delivery_setting_limit_distance,delivery_merchant_time,delivery_prepare_duration,delivery_ship_types,delivery_avg_price,delivery_avg_price_unit,delivery_shipping_fee_value,delivery_shipping_fee_minimum,delivery_shipping_fee_rate,delivery_min_charge,is_display_cutlery,confirm_methods,name_en,foody_service_id,min_order_value,root_category_ids,promotion_count,price_range_min_price,price_range_max_price,parent_category_id,position_verifie,total_like,total_order_y
0,0,62676,ho-chi-minh,0,['Vietnam'],"582 Quốc Lộ 13, P. Hiệp Bình Phước, Thủ Đức, T...",Liên Hoa - Gà Ác & Óc Heo Tiềm Thuốc Bắc,lien-hoa-ga-ac-oc-heo-tiem-thuoc-bac,10.850111,106.719257,693,786442,True,NaN,NaN,NaN,NaN,NaN,20000,Bistro,False,62676,0,10,4.8,10+,foody://foody.vn?action=reviewlist&id=786442,False,False,True,2,"[{""note"": {""text"": {""message"": ""This fee appli...",NaN,NaN,1,0,[],ShopeeFood,0,-1,"[6, 1, 4, 8]",False,20000,12,12,[],55000,đ,6500,"15,000đ",1.0,"3,000đ",True,{},NaN,1,20000,"[2387, 2437, 2438]",1,45000,60000,NaN,False,0.0,343.0
1,1,33938,ho-chi-minh,0,"['Vietnam', 'International']","11 Nguyễn Văn Trỗi, P. 12, Phú Nhuận, TP. HCM",NS Bến Thuyền,ngoc-suong-ben-thuyen-nguyen-van-troi,10.791926,106.681183,17,14,True,NaN,NaN,NaN,NaN,NaN,20000,Restaurant*#*Luxury,False,33938,0,100,4.8,100+,foody://foody.vn?action=reviewlist&id=14,False,False,False,2,"[{""note"": {""text"": {""message"": ""This fee appli...",NaN,NaN,1,0,"[{""start_time"": ""10:00"", ""week_day"": 1, ""end_t...",ShopeeFood,0,-1,"[6, 1, 4, 8]",False,20000,12,12,[],71839,đ,6500,"15,000đ",1.0,"3,000đ",True,{},Ngoc Suong Restaurant,1,20000,"[2419, 2546, 2387, 2395]",1,200000,1100000,2546.0,True,6.0,597.0
2,2,8077,ho-chi-minh,0,['Vietnam'],"15F Nguyễn Thị Minh Khai, P. Bến Nghé, Quận 1,...",Passio Coffee - 15F Nguyễn Thị Minh Khai,passio-coffee-15f-nguyen-thi-minh-khai,10.786102,106.701135,1,50,True,NaN,282.0,passio-coffee,Passio Coffee To Go,76.0,20000,Coffee/Dessert,False,8077,0,1000,4.9,999+,foody://foody.vn?action=reviewlist&id=50,False,False,False,1,[],NaN,NaN,1,0,"[{""start_time"": ""07:00"", ""week_day"": 1, ""end_t...",ShopeeFood,0,-1,"[6, 1, 4, 8]",True,20000,12,12,[],36467,đ,6500,"15,000đ",1.0,"3,000đ",True,{},Passio Coffee,1,20000,"[2448, 2450, 2388, 2455]",17,30000,55000,2448.0,True,1071.0,39627.0
3,3,69508,ho-chi-minh,0,['Vietnam'],"89 Trần Quý, P. 4, Quận 11, TP. HCM",Bún Mì Vàng & Hủ Tiếu Mì Thập Cẩm,bun-mi-vang-hu-tieu-mi-thap-cam,10.758493,106.653401,13,786494,True,NaN,NaN,NaN,NaN,NaN,20000,Bistro,False,69508,0,10,4.8,10+,foody://foody.vn?action=reviewlist&id=786494,False,False,True,2,"[{""note"": {""text"": {""message"": ""This fee appli...",NaN,NaN,1,0,"[{""start_time"": ""07:00"", ""week_day"": 1, ""end_t...",ShopeeFood,0,-1,"[6, 1, 4, 8]",False,20000,12,12,[],50800,đ,6500,"15,000đ",1.0,"3,000đ",True,{},NaN,1,20000,"[2426, 2387]",1,38000,46000,2387.0,False,6.0,824.0
4,4,8275,ho-chi-minh,0,"['South', 'Southeast']","261 - 263 An Dương Vương, P. 3, Quận 5, TP. HCM",Hải Sản Bê Vàng,hai-san-be-vang,10.759281,106.678682,7,73,True,NaN,NaN,NaN,NaN,NaN,20000,Restaurant,False,8275,0,50,4.9,50+,foody://foody.vn?action=reviewlist&id=73,False,False,True,2,"[{""note"": {""text"": {""message"": ""This fee appli...",NaN,NaN,1,0,"[{""start_time"": ""08:00"", ""week_day"": 1, ""end_t...",ShopeeFood,0,-1,"[6, 1, 4, 8]",False,20000,12,12,[],85600,đ,6500,"15,000đ",1.0,"3,00

## Prepocess

In [15]:
# drop
shopee_df=shopee_df\
.drop(columns=['Unnamed: 0','browsing_id','location_url','district_id','is_favorite','total_order_x','rating_display_total_review'\
               ,'rating_app_link','is_subscribe','vat','confirm_language','service_type','user_favorite_count','delivery_service_fee'\
               ,'delivery_ship_types','delivery_avg_price_unit','delivery_shipping_fee_value','delivery_min_charge','confirm_methods'\
               ])
# rename
shopee_df.rename(columns={'total_order_y':'total_order'},inplace=True)

In [16]:
#address
city=[]
district=[]
detail=[]
for row in shopee_df['address'].str.split(','):
  city.append(row[-1].strip())
  district.append(row[-2].strip())
  detail.append(','.join(row[0:-2]).strip())
shopee_df['address_detail']=detail
shopee_df['address_district']=district
shopee_df['address_city']=city

In [17]:
#remove đ
shopee_df['delivery_shipping_fee_minimum']=shopee_df['delivery_shipping_fee_minimum'].str.split('đ',expand=True)[0].str.replace(',','').astype(float)

In [18]:

# arrange
shopee_df=shopee_df[[
           # basic info
            'restaurant_id','restaurant_url','restaurant_name','name_en','restaurant_short_description','address_detail','address_district','address_city','lat', 'lon'\
           # 1:n cardinality
           ,'categories','cuisines','root_category_ids','delivery_week_days'\
           # 1:n brand's columns
           , 'brand_id', 'brand_url', 'brand_name','restaurant_count'\
           # delivery's columns
           ,'delivery_avg_price',  'delivery_fees',  'delivery_has_contract',  'delivery_id',  'delivery_merchant_limit_distance'\
           ,'delivery_merchant_time',  'delivery_payment_methods',  'delivery_prepare_duration',  'delivery_service_by',  'delivery_setting_limit_distance'\
           ,'delivery_shipping_fee_minimum',  'delivery_shipping_fee_rate','limit_distance'\
           # boolean columns
           ,'asap_is_available', 'has_phone',  'is_city_alert',  'is_display_cutlery',  'is_quality_merchant',  'contract_type',  'position_verifie'\
           # other id
           ,'foody_service_id', 'parent_category_id'\
           # numeric
           ,'min_order_value',  'price_range_max_price',  'price_range_min_price',  'total_like',  'total_order', 'rating_avg',  'rating_total_review', 'promotion_count'\
           ]]

In [19]:
shopee_df.dtypes

restaurant_id                         int64
restaurant_url                       object
restaurant_name                      object
name_en                              object
restaurant_short_description         object
address_detail                       object
address_district                     object
address_city                         object
lat                                 float64
lon                                 float64
categories                           object
cuisines                             object
root_category_ids                    object
delivery_week_days                   object
brand_id                            float64
brand_url                            object
brand_name                           object
restaurant_count                    float64
delivery_avg_price                    int64
delivery_fees                        object
delivery_has_contract                  bool
delivery_id                           int64
delivery_merchant_limit_distance

In [20]:
shopee_df.to_excel('drive/MyDrive/HCMUS/2022 - 2023/Data/1 - raw data/summary/cleaned_shopee_restaurant.xlsx')

#Foody data

In [ ]:
# !ls drive/MyDrive/HCMUS/'2022 - 2023'/Data/'1 - raw data'/foody/hcmc-foody-0-101000

hcmc-0-101000.csv	       hcmc-0-101000-update-2.csv
hcmc-0-101000-update-1.csv     hcmc-0-101000-update-3.csv
hcmc-0-101000-update-1.gsheet  url-hcmc-0-101000.csv


In [31]:
foody_df=pd.read_csv('drive/MyDrive/HCMUS/2022 - 2023/Data/1 - raw data/foody/hcmc-foody-0-101000/hcmc-0-101000-update-3.csv').dropna(how= 'all')

In [32]:
foody_df.dtypes

latitude                  float64
longitude                 float64
restaurant_id             float64
district_id               float64
city_id                   float64
name                       object
street_address             object
area_address               object
district_address           object
region_address             object
category                   object
cuisine                    object
audience                   object
average_rating            float64
position_rating            object
price_rating               object
quality_rating             object
service_rating             object
space_rating               object
review_count              float64
review_excellent_count    float64
review_good_count         float64
review_average_count      float64
review_bad_count          float64
view_count                float64
checked_in_count          float64
favourite_count           float64
wanted_count              float64
picture_count             float64
status        

In [33]:
foody_df

,latitude,longitude,restaurant_id,district_id,city_id,name,street_address,area_address,district_address,region_address,category,cuisine,audience,average_rating,position_rating,price_rating,quality_rating,service_rating,space_rating,review_count,review_excellent_count,review_good_count,review_average_count,review_bad_count,view_count,checked_in_count,favourite_count,wanted_count,picture_count,status,time_clock,price_range,url
0,10.785022,106.698976,6.780640e+05,1.0,217.0,Bún Miến Gà Vỉa Hè,"37 Mạc Đĩnh Chi, P. Đa Kao",Khu vực Đài Truyền Hình,Quận 1,TP. HCM,Quán ăn,Món Việt,"Gia đình, Nhóm hội, Giới văn phòng",8.0,7.5,9.0,8.5,8.5,6.5,2.0,0.0,2.0,0.0,0.0,237.0,0.0,2.0,6.0,7.0,Đang hoạt động,11:00 - 13:00,20.000đ - 25.000đ,http://www.foody.vn/ho-chi-minh/bun-mien-ga-vi...
1,10.771073,106.703596,2.104930e+05,1.0,217.0,Thiên Nhiên - Vịt Quay Heo Quay,56 Hàm Nghi,Khu vực Bitexco Tower,Quận 1,TP. HCM,Quán ăn,Món Việt,"Gia đình, Nhóm hội, Giới văn phòng",8.0,10.0,10.0,10.0,5.0,5.0,1.0,0.0,1.0,0.0,0.0,635.0,1.0,1.0,3.0,2.0,Đang hoạt động,08:30 - 22:00,50.000đ - 200.000đ,http://www.foody.vn/ho-chi-minh/thien-nhien-vi...
2,10.769882,106.691158,1.059918e+06,1.0,217.0,Tokyo Ramen,"106 Nguyễn Trãi, P. Bến Thành",Khu vực Zen Plaza,Quận 1,TP. HCM,Quán ăn,Món Nhật,"Gia đình, Nhóm hội, Giới văn phòng",8.0,7.0,8.0,9.0,8.0,8.0,1.0,0.0,1.0,0.0,0.0,168.0,0.0,1.0,0.0,46.0,Đang hoạt động,09:00 - 20:00,20.000đ - 200.000đ,http://www.foody.vn/ho-chi-minh/tokyo-ramen
3,10.793500,106.688799,1.043820e+06,1.0,217.0,Bún Bò Huế O Hương - Trần Quý Khoách,"10 Trần Quý Khoách, P. Tân Định",Khu vực Chợ Tân Định,Quận 1,TP. HCM,Quán ăn,Món Việt,"Sinh viên, Gia đình, Nhóm hội",8.0,10.0,10.0,9.0,5.0,6.0,1.0,0.0,1.0,0.0,0.0,136.0,0.0,3.0,2.0,12.0,Đang hoạt động,06:00 - 11:00,30.000đ - 50.000đ,http://www.foody.vn/ho-chi-minh/bun-bo-hue-o-h...
4,10.763958,106.697598,1.044890e+06,1.0,217.0,Lẩu Mini Hotpot,1 Nguyễn Thái Học,Khu vực Cầu Ông Lãnh,Quận 1,TP. HCM,Quán ăn,Món Việt,"Gia đình, Nhóm hội, Giới văn phòng",8.0,8.0,7.0,9.0,9.0,7.0,1.0,0.0,1.0,0.0,0.0,474.0,0.0,2.0,1.0,10.0,Đang hoạt động,10:00 - 23:00,50.000đ - 200.000đ,http://www.foody.vn/ho-chi-minh/lau-mini-hotpot
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101396,10.848723,106.759121,3.172930e+05,693.0,217.0,Sinh Tố & Bánh Mì Thổ Nhĩ Kỳ,"241 Đặng Văn Bi, P. Trường Thọ",Khu vực Làng Cafe,Quận Thủ Đức,TP. HCM,Ăn vặt/vỉa hè,Mỹ,"Sinh viên, Giới văn phòng",8.6,8.5,9.0,9.0,8.5,8.0,2.0,0.0,2.0,0.0,0.0,592.0,1.0,4.0,13.0,9.0,Đang hoạt động,05:00 - 22:00,10.000đ - 20.000đ,http://www.foody.vn/ho-chi-minh/sinh-to-banh-m...
101397,10.841441,106.825420,1.000030e+09,11.0,217.0,Sơn Dừa - Bánh Cuốn Nóng & Bún Thịt Nướ...,"6 Nguyễn Văn Thạnh, P. Long Thạnh Mỹ",NaN,Quận 9,TP. HCM,Ăn vặt/vỉa hè,"Món Bắc, Món Miền Nam","Sinh viên, Cặp đôi, Gia đình, Nhóm hội, Giới v...",NaN,--,--,--,--,--,0.0,NaN,NaN,NaN,NaN,39.0,0.0,0.0,0.0,0.0,Đang hoạt động,06:00 - 11:00,Đang cập nhật,http://www.foody.vn/ho-chi-minh/son-du-a-ba-nh...
101398,10.838521,106.831660,1.152490e+06,11.0,217.0,Highlands Coffee - Vinhomes Grand Park,"Vinhomes Grand Park, Nguyễn Xiển, Long Thạnh M...",NaN,Quận 9,TP. HCM,Café/Dessert,NaN,NaN,NaN,--,--,--,--,--,0.0,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,Đang hoạt động,07:00 - 22:00,30.000đ - 80.000đ,http://www.foody.vn/ho-chi-minh/highlands-coff...
101399,10.786297,106.753080,1.126699e+06,4.0,217.0,Nhà Hàng Mix Cafe - Cơm Gà Nướng & Steak,"36 Đường Số 2, P. An Phú (Ngay Góc Thân Văn Nh...",NaN,Quận 2,TP. HCM,Nhà hàng,NaN,NaN,NaN,--,--,--,--,--,0.0,NaN,NaN,NaN,NaN,42.0,0.0,0.0,0.0,0.0,Đang hoạt động,07:00 - 22:00,20.000đ - 100.000đ,http://www.foody.vn/ho-chi-minh/nha-hang-mix-c...


## Preprocess

In [34]:
# url
foody_df['url']=foody_df['url'].str.split('/',expand=True)[4]

#full_address
foody_df['full_address']= foody_df['street_address'] +', '+ foody_df['district_address'] +', '+ foody_df['region_address']

# price range
price_range_temp=foody_df['price_range'].str.split('-',expand=True)

min_price_temp=price_range_temp[0]
min_price_temp=min_price_temp.replace('Đang cập nhật',np.nan)
min_price_temp=min_price_temp\
              .str.replace('đ','')\
              .str.replace('.','')\
              .str.strip().astype(float)

max_price_temp=price_range_temp[1]
max_price_temp=max_price_temp.replace('None',np.nan)
max_price_temp=max_price_temp\
              .str.replace('đ','')\
              .str.replace('.','')\
              .str.strip().astype(float)
foody_df['min_price']=min_price_temp
foody_df['max_price']=max_price_temp

foody_df['restaurant_id']=[int(x) for x in foody_df['restaurant_id']]
foody_df.drop(columns=['district_id','city_id','price_range'],inplace=True)

<ipython-input-34-df61f28852b8>:12: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  min_price_temp=min_price_temp\
<ipython-input-34-df61f28852b8>:19: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  max_price_temp=max_price_temp\


In [35]:
#arrange
foody_df=foody_df[[
                  # basic info
                    'restaurant_id','name','url','full_address','area_address','street_address','district_address','region_address','latitude','longitude'\
                  # 1:n cardinality
                  ,'audience', 'category', 'cuisine'
                  # 
                  ,'time_clock', 'status', 'min_price','max_price'\
                  # rating
                  ,'average_rating', 'position_rating',  'price_rating',  'quality_rating',  'service_rating',  'space_rating'\
                  # count
                  ,'checked_in_count', 'favourite_count', 'review_average_count',  'review_bad_count',  'review_count',  'review_excellent_count'\
                  ,'review_good_count',    'view_count',  'wanted_count',  'picture_count'
                  ]]

In [36]:
foody_df.to_excel('drive/MyDrive/HCMUS/2022 - 2023/Data/1 - raw data/summary/cleaned_foody_restaurant.xlsx')

# Aggregate

In [37]:
shopee_df.head(3)

,restaurant_id,restaurant_url,restaurant_name,name_en,restaurant_short_description,address_detail,address_district,address_city,lat,lon,categories,cuisines,root_category_ids,delivery_week_days,brand_id,brand_url,brand_name,restaurant_count,delivery_avg_price,delivery_fees,delivery_has_contract,delivery_id,delivery_merchant_limit_distance,delivery_merchant_time,delivery_payment_methods,delivery_prepare_duration,delivery_service_by,delivery_setting_limit_distance,delivery_shipping_fee_minimum,delivery_shipping_fee_rate,limit_distance,asap_is_available,has_phone,is_city_alert,is_display_cutlery,is_quality_merchant,contract_type,position_verifie,foody_service_id,parent_category_id,min_order_value,price_range_max_price,price_range_min_price,total_like,total_order,rating_avg,rating_total_review,promotion_count
0,786442,lien-hoa-ga-ac-oc-heo-tiem-thuoc-bac,Liên Hoa - Gà Ác & Óc Heo Tiềm Thuốc Bắc,NaN,NaN,"582 Quốc Lộ 13, P. Hiệp Bình Phước",Thủ Đức,TP. HCM,10.850111,106.719257,Bistro,['Vietnam'],"[2387, 2437, 2438]",[],NaN,NaN,NaN,NaN,55000,"[{""note"": {""text"": {""message"": ""This fee appli...",False,62676,-1,12,"[6, 1, 4, 8]",12,ShopeeFood,20000,15000.0,1.0,20000,False,True,True,True,0,2,False,1,NaN,20000,60000,45000,0.0,343.0,4.8,10,1
1,14,ngoc-suong-ben-thuyen-nguyen-van-troi,NS Bến Thuyền,Ngoc Suong Restaurant,NaN,"11 Nguyễn Văn Trỗi, P. 12",Phú Nhuận,TP. HCM,10.791926,106.681183,Restaurant*#*Luxury,"['Vietnam', 'International']","[2419, 2546, 2387, 2395]","[{""start_time"": ""10:00"", ""week_day"": 1, ""end_t...",NaN,NaN,NaN,NaN,71839,"[{""note"": {""text"": {""message"": ""This fee appli...",False,33938,-1,12,"[6, 1, 4, 8]",12,ShopeeFood,20000,15000.0,1.0,20000,False,True,False,True,0,2,True,1,2546.0,20000,1100000,200000,6.0,597.0,4.8,100,1
2,50,passio-coffee-15f-nguyen-thi-minh-khai,Passio Coffee - 15F Nguyễn Thị Minh Khai,Passio Coffee,NaN,"15F Nguyễn Thị Minh Khai, P. Bến Nghé",Quận 1,TP. HCM,10.786102,106.701135,Coffee/Dessert,['Vietnam'],"[2448, 2450, 2388, 2455]","[{""start_time"": ""07:00"", ""week_day"": 1, ""end_t...",282.0,passio-coffee,Passio Coffee To Go,76.0,36467,[],True,8077,-1,12,"[6, 1, 4, 8]",12,ShopeeFood,20000,15000.0,1.0,20000,False,True,False,True,0,1,True,1,2448.0,20000,55000,30000,1071.0,39627.0,4.9,1000,17


In [38]:
foody_df.head(3)

,restaurant_id,name,url,full_address,area_address,street_address,district_address,region_address,latitude,longitude,audience,category,cuisine,time_clock,status,min_price,max_price,average_rating,position_rating,price_rating,quality_rating,service_rating,space_rating,checked_in_count,favourite_count,review_average_count,review_bad_count,review_count,review_excellent_count,review_good_count,view_count,wanted_count,picture_count
0,678064,Bún Miến Gà Vỉa Hè,bun-mien-ga-via-he,"37 Mạc Đĩnh Chi, P. Đa Kao, Quận 1, TP. HCM",Khu vực Đài Truyền Hình,"37 Mạc Đĩnh Chi, P. Đa Kao",Quận 1,TP. HCM,10.785022,106.698976,"Gia đình, Nhóm hội, Giới văn phòng",Quán ăn,Món Việt,11:00 - 13:00,Đang hoạt động,20000.0,25000.0,8.0,7.5,9.0,8.5,8.5,6.5,0.0,2.0,0.0,0.0,2.0,0.0,2.0,237.0,6.0,7.0
1,210493,Thiên Nhiên - Vịt Quay Heo Quay,thien-nhien-vit-quay-heo-quay,"56 Hàm Nghi, Quận 1, TP. HCM",Khu vực Bitexco Tower,56 Hàm Nghi,Quận 1,TP. HCM,10.771073,106.703596,"Gia đình, Nhóm hội, Giới văn phòng",Quán ăn,Món Việt,08:30 - 22:00,Đang hoạt động,50000.0,200000.0,8.0,10.0,10.0,10.0,5.0,5.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,635.0,3.0,2.0
2,1059918,Tokyo Ramen,tokyo-ramen,"106 Nguyễn Trãi, P. Bến Thành, Quận 1, TP. HCM",Khu vực Zen Plaza,"106 Nguyễn Trãi, P. Bến Thành",Quận 1,TP. HCM,10.769882,106.691158,"Gia đình, Nhóm hội, Giới văn phòng",Quán ăn,Món Nhật,09:00 - 20:00,Đang hoạt động,20000.0,200000.0,8.0,7.0,8.0,9.0,8.0,8.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,168.0,0.0,46.0


## Only shopee

In [39]:
shopee_only=set(shopee_df['restaurant_id'])-set(foody_df['restaurant_id'])
len(shopee_only)

20082

In [41]:
shopee_df[shopee_df['restaurant_id'].isin(list(shopee_only))].to_excel('drive/MyDrive/HCMUS/2022 - 2023/Data/1 - raw data/summary/shopee_only.xlsx')

## Only foody

In [42]:
foody_only=set(foody_df['restaurant_id'])-set(shopee_df['restaurant_id'])
len(foody_only)

39817

In [43]:
foody_df[foody_df['restaurant_id'].isin(list(foody_only))].to_excel('drive/MyDrive/HCMUS/2022 - 2023/Data/1 - raw data/summary/foody_only.xlsx')

## Both

In [44]:
both_shopee_foody=set(shopee_df['restaurant_id']).intersection(set(foody_df['restaurant_id']))
len(both_shopee_foody)

61581

In [45]:
both_shopee_df=shopee_df[shopee_df['restaurant_id'].isin(list(both_shopee_foody))]

In [46]:
both_shopee_df.shape

(61581, 48)

In [47]:
both_shopee_df.head(3)

,restaurant_id,restaurant_url,restaurant_name,name_en,restaurant_short_description,address_detail,address_district,address_city,lat,lon,categories,cuisines,root_category_ids,delivery_week_days,brand_id,brand_url,brand_name,restaurant_count,delivery_avg_price,delivery_fees,delivery_has_contract,delivery_id,delivery_merchant_limit_distance,delivery_merchant_time,delivery_payment_methods,delivery_prepare_duration,delivery_service_by,delivery_setting_limit_distance,delivery_shipping_fee_minimum,delivery_shipping_fee_rate,limit_distance,asap_is_available,has_phone,is_city_alert,is_display_cutlery,is_quality_merchant,contract_type,position_verifie,foody_service_id,parent_category_id,min_order_value,price_range_max_price,price_range_min_price,total_like,total_order,rating_avg,rating_total_review,promotion_count
0,786442,lien-hoa-ga-ac-oc-heo-tiem-thuoc-bac,Liên Hoa - Gà Ác & Óc Heo Tiềm Thuốc Bắc,NaN,NaN,"582 Quốc Lộ 13, P. Hiệp Bình Phước",Thủ Đức,TP. HCM,10.850111,106.719257,Bistro,['Vietnam'],"[2387, 2437, 2438]",[],NaN,NaN,NaN,NaN,55000,"[{""note"": {""text"": {""message"": ""This fee appli...",False,62676,-1,12,"[6, 1, 4, 8]",12,ShopeeFood,20000,15000.0,1.0,20000,False,True,True,True,0,2,False,1,NaN,20000,60000,45000,0.0,343.0,4.8,10,1
1,14,ngoc-suong-ben-thuyen-nguyen-van-troi,NS Bến Thuyền,Ngoc Suong Restaurant,NaN,"11 Nguyễn Văn Trỗi, P. 12",Phú Nhuận,TP. HCM,10.791926,106.681183,Restaurant*#*Luxury,"['Vietnam', 'International']","[2419, 2546, 2387, 2395]","[{""start_time"": ""10:00"", ""week_day"": 1, ""end_t...",NaN,NaN,NaN,NaN,71839,"[{""note"": {""text"": {""message"": ""This fee appli...",False,33938,-1,12,"[6, 1, 4, 8]",12,ShopeeFood,20000,15000.0,1.0,20000,False,True,False,True,0,2,True,1,2546.0,20000,1100000,200000,6.0,597.0,4.8,100,1
2,50,passio-coffee-15f-nguyen-thi-minh-khai,Passio Coffee - 15F Nguyễn Thị Minh Khai,Passio Coffee,NaN,"15F Nguyễn Thị Minh Khai, P. Bến Nghé",Quận 1,TP. HCM,10.786102,106.701135,Coffee/Dessert,['Vietnam'],"[2448, 2450, 2388, 2455]","[{""start_time"": ""07:00"", ""week_day"": 1, ""end_t...",282.0,passio-coffee,Passio Coffee To Go,76.0,36467,[],True,8077,-1,12,"[6, 1, 4, 8]",12,ShopeeFood,20000,15000.0,1.0,20000,False,True,False,True,0,1,True,1,2448.0,20000,55000,30000,1071.0,39627.0,4.9,1000,17


In [48]:
both_foody_df=foody_df[foody_df['restaurant_id'].isin(list(both_shopee_foody))]

In [49]:
both_foody_df.shape

(61583, 33)

In [50]:
both_foody_df.head(3)

,restaurant_id,name,url,full_address,area_address,street_address,district_address,region_address,latitude,longitude,audience,category,cuisine,time_clock,status,min_price,max_price,average_rating,position_rating,price_rating,quality_rating,service_rating,space_rating,checked_in_count,favourite_count,review_average_count,review_bad_count,review_count,review_excellent_count,review_good_count,view_count,wanted_count,picture_count
2,1059918,Tokyo Ramen,tokyo-ramen,"106 Nguyễn Trãi, P. Bến Thành, Quận 1, TP. HCM",Khu vực Zen Plaza,"106 Nguyễn Trãi, P. Bến Thành",Quận 1,TP. HCM,10.769882,106.691158,"Gia đình, Nhóm hội, Giới văn phòng",Quán ăn,Món Nhật,09:00 - 20:00,Đang hoạt động,20000.0,200000.0,8.0,7.0,8.0,9.0,8.0,8.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,168.0,0.0,46.0
9,1074507,Clover Korean House Food - Phan Liêm,clover-korean-house-food-phan-liem,"2 Phan Liêm, P. Đakao, Quận 1, TP. HCM",NaN,"2 Phan Liêm, P. Đakao",Quận 1,TP. HCM,10.789082,106.695763,NaN,Quán ăn,NaN,09:00 - 21:40,Đang hoạt động,10000.0,445000.0,8.0,8.0,8.0,8.0,8.0,8.0,0.0,7.0,0.0,0.0,1.0,0.0,1.0,334.0,5.0,0.0
13,940085,Liên Hoa - Cơm Chiên & Mì Ý,lien-hoa-com-chien-mi-y,"168/11 Trần Đình Xu, P. Nguyễn Cư Trinh, Quận ...",Khu vực Citi Mart Nguyễn Trãi,"168/11 Trần Đình Xu, P. Nguyễn Cư Trinh",Quận 1,TP. HCM,10.763958,106.687953,"Gia đình, Nhóm hội, Giới văn phòng",Quán ăn,Món Việt,17:00 - 22:00,Đang hoạt động,30000.0,70000.0,7.9,10.0,5.0,9.5,10.0,5.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0,77.0,1.0,1.0


### Handle time

#### Explore

In [51]:
both_foody_df['time_clock']

2                         09:00 - 20:00
9                         09:00 - 21:40
13                        17:00 - 22:00
14        00:01 - 03:00 | 15:00 - 23:59
16                        06:00 - 14:30
                      ...              
101393                    11:00 - 21:00
101394                    10:00 - 22:00
101397                    06:00 - 11:00
101398                    07:00 - 22:00
101399                    07:00 - 22:00
Name: time_clock, Length: 61583, dtype: object

In [52]:
distinct_delivery_week_days=list(both_shopee_df['delivery_week_days'].value_counts().index)
distinct_delivery_week_days=distinct_delivery_week_days[1:]
distinct_delivery_week_days=[json.loads(x) for x in distinct_delivery_week_days]

Multi period a day

In [53]:
#check if multi period in a day
has_2_period=[]
for res in distinct_delivery_week_days:
  week_days=[0]*7
  for day in res:
    week_days[day['week_day']-1]+=1
    if week_days[day['week_day']-1] > 1:
      has_2_period.append(res)
      break
len(has_2_period)

2849

In [54]:
has_2_period[0]

[{'start_time': '06:00', 'week_day': 1, 'end_time': '13:00'},
 {'start_time': '06:00', 'week_day': 2, 'end_time': '13:00'},
 {'start_time': '06:00', 'week_day': 3, 'end_time': '13:00'},
 {'start_time': '06:00', 'week_day': 4, 'end_time': '13:00'},
 {'start_time': '06:00', 'week_day': 5, 'end_time': '13:00'},
 {'start_time': '06:00', 'week_day': 6, 'end_time': '13:00'},
 {'start_time': '06:00', 'week_day': 7, 'end_time': '13:00'},
 {'start_time': '16:00', 'week_day': 1, 'end_time': '21:00'},
 {'start_time': '16:00', 'week_day': 2, 'end_time': '21:00'},
 {'start_time': '16:00', 'week_day': 3, 'end_time': '21:00'},
 {'start_time': '16:00', 'week_day': 4, 'end_time': '21:00'},
 {'start_time': '16:00', 'week_day': 5, 'end_time': '21:00'},
 {'start_time': '16:00', 'week_day': 6, 'end_time': '21:00'},
 {'start_time': '16:00', 'week_day': 7, 'end_time': '21:00'}]

In [55]:
#check if days in week have the same period

Different day, different period

In [56]:
#different period at different day
different_period=[]
log=[]
for res in distinct_delivery_week_days:
  check_list=[[], [], [], [], [], [], []]

  for day in res:
    check_list[day['week_day']-1].append({'start_time':day['start_time'],'end_time':day['end_time']})
  
  stop_flag=0

  check_list=list(filter(lambda a: a != [], check_list))

  for x in check_list:
    if stop_flag==1:
      break 
    else:
      for y in check_list:
        if x!=y:
          different_period.append(res)
          log.append(check_list)
          stop_flag=1
          break
        

In [57]:
len(different_period)

11281


Different day, different period - many period a day

In [58]:
diff_multi_period=[]
for x in log:
  for y in x:
    if len(y)>1:
     diff_multi_period.append(x)
  

In [59]:
diff_multi_period[0]

[[{'start_time': '07:30', 'end_time': '12:45'},
  {'start_time': '13:15', 'end_time': '21:25'}],
 [{'start_time': '07:30', 'end_time': '12:45'},
  {'start_time': '13:15', 'end_time': '21:30'}],
 [{'start_time': '07:30', 'end_time': '12:45'},
  {'start_time': '13:15', 'end_time': '21:30'}],
 [{'start_time': '07:30', 'end_time': '12:45'},
  {'start_time': '13:15', 'end_time': '21:30'}],
 [{'start_time': '07:30', 'end_time': '12:45'},
  {'start_time': '13:15', 'end_time': '21:30'}],
 [{'start_time': '07:30', 'end_time': '12:45'},
  {'start_time': '13:15', 'end_time': '21:30'}],
 [{'start_time': '07:30', 'end_time': '12:45'},
  {'start_time': '13:15', 'end_time': '21:30'}]]

#### Process

In [60]:
def process_time_week_day(id,time_stamp):
  for row in time_stamp:
    
    start_time=row['start_time'].split(':')
    start_time=round(int(start_time[0]) + int(start_time[1])/60 + 0.01)
    
    end_time=row['end_time'].split(':')
    end_time=round(int(end_time[0]) + int(end_time[1])/60 + 0.01)

    for i in range(start_time,end_time):
      restaurant_id.append(id)
      week_day.append(row['week_day'])
      active_hour.append(i)



In [61]:
restaurant_id=[]
week_day=[]
active_hour=[]
for res in both_shopee_df[['restaurant_id','delivery_week_days']].iterrows():
  process_time_week_day(res[1][0],json.loads(res[1][1]))

In [62]:
active_hour_df=pd.DataFrame({'restaurant_id':restaurant_id,'week_day':week_day,'active_hour':active_hour})

In [64]:
active_hour_df.to_csv('drive/MyDrive/HCMUS/2022 - 2023/Data/1 - raw data/summary/active_hour.csv',index=False)

### Category

#### Explore

In [66]:
temp_df=both_shopee_df[['restaurant_id','categories','cuisines','root_category_ids']].merge(both_foody_df[['restaurant_id','audience','category','cuisine']],on=['restaurant_id'])

In [67]:
temp_df[['categories','category']].drop_duplicates()

,categories,category
0,Bistro,Quán ăn
1,Restaurant*#*Luxury,Sang trọng
2,Coffee/Dessert,Café/Dessert
4,Restaurant,Nhà hàng
5,Street food,Ăn vặt/vỉa hè
...,...,...
52039,Supermarket*#*Foodcourt,Siêu thị
54812,Bakery*#*Office lunch delivery,Tiệm bánh
55493,Coffee/Dessert*#*Sport Club,Café/Dessert
56013,Shop/Stores*#*Office lunch delivery,Ăn chay


In [68]:
cat_list=[]
for cat in temp_df['categories']:
  cat_list.extend(cat.split('*#*'))


In [69]:
len(set(cat_list))

37

In [70]:
set(cat_list)

{'Bakery',
 'Bar/Pub',
 'Beauty shop',
 'Beer club',
 'Billiards',
 'Bistro',
 'Boozer',
 'Buffet',
 'Catering',
 'Cinema',
 'Cleaning Service',
 'Coffee/Dessert',
 'Foodcourt',
 'Gaming Zone',
 'Homestay',
 'Karaoke',
 'Luxury',
 'Market',
 'Nail salon',
 'Office lunch delivery',
 'Online Shopping',
 'Pets',
 'Restaurant',
 'Shop',
 'Shop Online',
 'Shop/Stores',
 'Sight & Landmark',
 'Spa/Massage',
 'Sport Club',
 'Street food',
 'Supermarket',
 'Tattoo',
 'Theater',
 'University & College',
 'Vegetarian',
 'Wedding flowers',
 'Wedding/Convention'}

In [71]:
set([x.strip() for x in temp_df['category']])

{'Bar/Pub',
 'Beer club',
 'Buffet',
 'Café/Dessert',
 'Giao cơm văn phòng',
 'Khu Ẩm Thực',
 'Nhà hàng',
 'Quán nhậu',
 'Quán ăn',
 'Sang trọng',
 'Shop Online',
 'Shop hóa mỹ phẩm',
 'Shop/Cửa hàng',
 'Siêu thị',
 'Tiệc tận nơi',
 'Tiệm bánh',
 'Ăn chay',
 'Ăn vặt/vỉa hè'}

#### Process

In [72]:
#shopee
restaurant_id_list=[]
categories_list=[]
for row in both_shopee_df[['restaurant_id','categories']].iterrows():
  for item in row[1][1].split('*#*'):
    restaurant_id_list.append(row[1][0])
    categories_list.append(item.strip())

In [73]:
category_shopee_df=pd.DataFrame({'restaurant_id':restaurant_id_list,'category':categories_list }).sort_values(['restaurant_id'])

In [74]:
#foody
both_foody_df['category']=both_foody_df['category'].str.strip()

<ipython-input-74-b9d27ec6f2b7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  both_foody_df['category']=both_foody_df['category'].str.strip()


In [75]:
category_foody_df=both_foody_df[['restaurant_id','category']].copy()

### Cuisines

#### Explore

In [76]:
temp_df[['cuisines','cuisine']].drop_duplicates()

,cuisines,cuisine
0,['Vietnam'],Món Việt
1,"['Vietnam', 'International']","Món Việt, Quốc tế"
4,"['South', 'Southeast']",Miền Đông
6,[],NaN
30,"['Chinese', 'Asian']",Món Trung Hoa
...,...,...
60253,Vietnam*#*South,"Mỹ, Đặc sản vùng, Món Miền Nam, Tây Nguyên"
60510,North*#*Korean,"Món Bắc, Món Hàn"
61108,Vietnam*#*Thailand,"Món Việt, Món Nhật, Món Hàn, Món Thái, Singapore"
61333,Vietnam*#*Italian,"Món Việt, Món Á, Pháp, Ý, Đức"


#### Process

In [77]:
#shopee
temp=both_shopee_df[['restaurant_id','cuisines']].copy()
temp['cuisines']=temp['cuisines'].astype(str)
temp

,restaurant_id,cuisines
0,786442,['Vietnam']
1,14,"['Vietnam', 'International']"
2,50,['Vietnam']
3,786494,['Vietnam']
4,73,"['South', 'Southeast']"
...,...,...
81654,1083136,nan
81655,1000033541,Vietnam*#*South
81657,645072,Vietnam
81658,708242,Vietnam


In [78]:
restaurant_id_list=[]
cuisines_list=[]
for row in temp[~temp['cuisines'].str.contains(']')].iterrows():
    if row[1][1] != 'nan':
      for item in row[1][1].split('*#*'):
        restaurant_id_list.append(row[1][0])
        cuisines_list.append(item.strip())
for row in temp[temp['cuisines'].str.contains(']')].iterrows():
    if row[1][1] != 'nan':
      for item in json.loads(row[1][1].replace("'",'"')):
        restaurant_id_list.append(row[1][0])
        cuisines_list.append(item.strip())

In [79]:
cuisine_shopee_df=pd.DataFrame({'restaurant_id':restaurant_id_list,
              'cuisine':cuisines_list
              })

In [80]:
#foody
both_foody_df['cuisine']=both_foody_df['cuisine'].astype(str)

<ipython-input-80-0effbcf7da33>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  both_foody_df['cuisine']=both_foody_df['cuisine'].astype(str)


In [81]:
restaurant_id_list=[]
cuisine_list=[]
for row in both_foody_df[['restaurant_id','cuisine']].iterrows():
  if row[1][1]!='nan':
      for item in row[1][1].split(','):
        restaurant_id_list.append(row[1][0])
        cuisine_list.append(item.strip())

In [82]:
cuisine_foody_df=pd.DataFrame({'restaurant_id':restaurant_id_list,
                               'cuisine':cuisine_list
                               }).sort_values(['restaurant_id'])

### Root_cate & audience

#### Explore

In [83]:
temp_df[['root_category_ids','audience']]

,root_category_ids,audience
0,"[2387, 2437, 2438]","Gia đình, Nhóm hội, Giới văn phòng"
1,"[2419, 2546, 2387, 2395]","Gia đình, Khách du lịch"
2,"[2448, 2450, 2388, 2455]",NaN
3,"[2426, 2387]","Gia đình, Nhóm hội, Giới văn phòng"
4,"[2419, 2546, 2387, 2426]",NaN
...,...,...
61578,"[2546, 2387]",NaN
61579,"[2426, 2387]","Sinh viên, Cặp đôi, Gia đình, Nhóm hội, Giới v..."
61580,"[2531, 2437, 2414, 2387, 2489, 2494]","Gia đình, Nhóm hội, Giới văn phòng"
61581,"[2448, 2388, 2455]","Cặp đôi, Nhóm hội, Giới văn phòng"


In [84]:
root_cate_list=[]

In [85]:
for root_cat in temp_df['root_category_ids']:
 root_cate_list.extend(json.loads(root_cat))

In [86]:
len(set(root_cate_list))

152

In [87]:
audience_list=[]
for au in temp_df['audience']:
  if type(au)==str:
    row=[x.strip() for x in au.split(',')]
    audience_list.extend(row)

In [88]:
set(audience_list)

{'Cặp đôi',
 'Gia đình',
 'Giới Manager',
 'Giới văn phòng',
 'Khách du lịch',
 'Nhóm hội',
 'Sinh viên',
 'Trẻ em'}

#### Process

In [89]:
#shopee
restaurant_id_list=[]
root_category_ids_list=[]
for row in both_shopee_df[['restaurant_id','root_category_ids']].iterrows():
  for item in json.loads(row[1][1]):
    restaurant_id_list.append(row[1][0])
    root_category_ids_list.append(item)

In [90]:
root_category_shopee_df=pd.DataFrame({
    'restaurant_id':restaurant_id_list,
    'root_category_ids':root_category_ids_list
}).sort_values(['restaurant_id'])

In [91]:
#foody
both_foody_df['audience']=both_foody_df['audience'].astype(str)

<ipython-input-91-d67c895b55a1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  both_foody_df['audience']=both_foody_df['audience'].astype(str)


In [92]:
both_foody_df[['restaurant_id','audience']]

,restaurant_id,audience
2,1059918,"Gia đình, Nhóm hội, Giới văn phòng"
9,1074507,nan
13,940085,"Gia đình, Nhóm hội, Giới văn phòng"
14,1033536,"Gia đình, Nhóm hội, Giới văn phòng"
16,762984,"Sinh viên, Cặp đôi, Nhóm hội, Giới văn phòng"
...,...,...
101393,1055482,"Sinh viên, Gia đình, Nhóm hội"
101394,863533,"Gia đình, Nhóm hội, Giới văn phòng"
101397,1000030101,"Sinh viên, Cặp đôi, Gia đình, Nhóm hội, Giới v..."
101398,1152490,nan


In [93]:
restaurant_id_list=[]
audience_list=[]
for row in both_foody_df[['restaurant_id','audience']].iterrows():
  if row[1][1]!='nan':
    for item in row[1][1].split(','):
      restaurant_id_list.append(row[1][0])
      audience_list.append(item.strip())

In [94]:
audience_foody_df=pd.DataFrame({
     'restaurant_id':restaurant_id_list,
     'audience':audience_list

}).sort_values(['restaurant_id'])

### Brand

#### Explore

In [95]:
distinct_brand=both_shopee_df[['brand_id','brand_url','brand_name','restaurant_count']].dropna(how='all').drop_duplicates()

In [96]:
distinct_brand[distinct_brand['brand_url']=='am-thuc-phan-rang']

,brand_id,brand_url,brand_name,restaurant_count
16485,NaN,am-thuc-phan-rang,NaN,2.0
69288,16461.0,am-thuc-phan-rang,ẨM THỰC PHAN RANG,2.0


In [97]:
duplicate_brand_url=list(distinct_brand['brand_url'].value_counts()[distinct_brand['brand_url'].value_counts()>1].index)

In [98]:
duplicate_brand_url

['am-thuc-phan-rang',
 'tra-sua-te-amo',
 'tra-sua-sunday-basic',
 'sup-cua-phuong',
 '2go-drinks-nuoc-mia-nuoc-ep',
 'bun-bo-hue-149',
 'us-coffee',
 'chao-trang-la-dua',
 'pasteur-street-brewing',
 'amigo-coffee',
 'tra-sua-nong',
 'tien-vua-food-sup-bao-ngu-vi-ca-hai-sam-vip',
 'com-tam-minh-long',
 'napoli-coffee',
 'pho-trang',
 'banh-mi-oi-hcm',
 'quan-com-sach',
 'savoure',
 'bap-xao-co-chanh',
 'hu-tieu-muc-ong-gia-cali',
 'sua-tuoi-tran-chau-duong-den',
 'big-seafood-sup-ba-ngu-vi-ca-tam-bo',
 'viva-star-coffee',
 'com-tam-phuc-loc-tho',
 'lien-huong-ga-ac-oc-heo-tiem-thuoc-bac',
 'chao-dinh-duong-viet-soup',
 'king-coffee',
 'com-tam-3-chi-em',
 'gach-cafe',
 'hong-tra-ngo-gia',
 'trai-cay-chi-thao',
 'sushi-88',
 'a-neo-tra-sua-an-vat']

In [99]:
distinct_brand[distinct_brand['brand_url']=='tra-sua-te-amo']

,brand_id,brand_url,brand_name,restaurant_count
543,8282.0,tra-sua-te-amo,Trà Sữa Te Amo,97.0
8697,8282.0,tra-sua-te-amo,Trà Sữa Te Amo_foody,97.0


In [100]:
distinct_brand[distinct_brand['brand_url'].isin(duplicate_brand_url)].sort_values(['brand_id']).to_excel('duplicate_brand.xlsx',index=False)


In [101]:
distinct_brand[distinct_brand['brand_url'].isin(duplicate_brand_url)].sort_values(['brand_id']).head(7)

,brand_id,brand_url,brand_name,restaurant_count
71757,117.0,savoure,Savouré Bakery,7.0
13311,117.0,savoure,Savouré Bakery,6.0
1729,720.0,viva-star-coffee,Viva Star Coffee,32.0
11668,720.0,viva-star-coffee,Viva Star Coffee_foody,32.0
8563,889.0,com-tam-phuc-loc-tho,Cơm Tấm Phúc Lộc Thọ,38.0
2606,889.0,com-tam-phuc-loc-tho,Cơm Tấm Phúc Lộc Thọ,39.0
10108,977.0,hu-tieu-muc-ong-gia-cali,Hủ Tiếu Mực Ông Già Cali_foody,3.0


In [102]:
duplicate_brand=list(distinct_brand['brand_url'].value_counts()[distinct_brand['brand_url'].value_counts()>1].index)

In [103]:
both_shopee_df[both_shopee_df['brand_id']==10725]

,restaurant_id,restaurant_url,restaurant_name,name_en,restaurant_short_description,address_detail,address_district,address_city,lat,lon,categories,cuisines,root_category_ids,delivery_week_days,brand_id,brand_url,brand_name,restaurant_count,delivery_avg_price,delivery_fees,delivery_has_contract,delivery_id,delivery_merchant_limit_distance,delivery_merchant_time,delivery_payment_methods,delivery_prepare_duration,delivery_service_by,delivery_setting_limit_distance,delivery_shipping_fee_minimum,delivery_shipping_fee_rate,limit_distance,asap_is_available,has_phone,is_city_alert,is_display_cutlery,is_quality_merchant,contract_type,position_verifie,foody_service_id,parent_category_id,min_order_value,price_range_max_price,price_range_min_price,total_like,total_order,rating_avg,rating_total_review,promotion_count
5712,1042061,a-neo-tra-sua-an-vat-204-kenh-19-thang-5b,Cơm Tấm 202,NaN,NaN,"204 Kênh 19 Tháng 5B, P. Tây Thạnh",Tân Phú,TP. HCM,10.813088,106.618612,Bistro,['Vietnam'],"[2387, 2414]","[{""start_time"": ""00:00"", ""week_day"": 3, ""end_t...",10725.0,a-neo-tra-sua-an-vat,A Nèo - Trà Sữa & Ăn Vặt,2.0,15607,[],True,129487,-1,12,"[6, 1, 4, 8]",12,ShopeeFood,20000,15000.0,1.00,20000,True,True,False,True,0,1,False,1,2414.0,20000,35000,5000,14.0,955.0,4.4,10,22
75520,1026083,a-neo-tra-sua-an-vat,A Nèo - Trà Sữa & Ăn Vặt - Kênh 19 Tháng 5B,NaN,NaN,"202 Kênh 19 Tháng 5B, P. Tây Thạnh",Tân Phú,TP. HCM,10.813091,106.618610,Coffee/Dessert,Vietnam,"[2448, 2388, 2455]","[{""start_time"": ""00:00"", ""week_day"": 1, ""end_t...",10725.0,a-neo-tra-sua-an-vat,Cơm Tấm 202,2.0,16828,[],True,117030,-1,12,"[6, 1, 4, 8]",12,ShopeeFood,7000,16000.0,1.02,20000,False,True,False,True,0,1,False,1,2448.0,20000,35000,5000,148.0,3671.0,4.5,50,14


#### Process

In [104]:
correct_duplicate_brand=pd.read_excel('drive/MyDrive/HCMUS/2022 - 2023/Data//1 - raw data/summary/old/correct_duplicate_brand.xlsx')
duplicate_brand_url=list(correct_duplicate_brand['brand_url'])

In [105]:
distinct_brand=both_shopee_df[['brand_id','brand_url','brand_name','restaurant_count']].dropna(how='all').drop_duplicates()
distinct_brand=distinct_brand[~distinct_brand['brand_id'].isna()]
distinct_brand=distinct_brand[~distinct_brand['brand_url'].isin(duplicate_brand_url)]

In [106]:
brand_shopee_df=distinct_brand.append(correct_duplicate_brand)

In [107]:
brand_shopee_df['brand_id']=brand_shopee_df['brand_id'].astype(int)
brand_shopee_df['restaurant_count']=brand_shopee_df['restaurant_count'].astype(int)

In [108]:
brand_shopee_df

,brand_id,brand_url,brand_name,restaurant_count
2,282,passio-coffee,Passio Coffee To Go,76
9,11148,chillsa-bread,Chillsa Bread,2
33,5605,com-ga-sot-me-sot-thai,Cơm Gà Sốt Me Sốt Thái,2
38,53,banh-mi-nhu-lan,Bánh Mì Như Lan,2
45,261,thai-village,Thai Village,1
...,...,...,...,...
26,12334,bun-bo-hue-149,Bún Bò Huế 149,3
27,14511,banh-mi-oi-hcm,Bánh Mì Ơi - HCM,23
28,15068,big-seafood-sup-ba-ngu-vi-ca-tam-bo,Big Seafood - Súp Bào Ngư Vi Cá Tẩm Bổ,4
29,15541,2go-drinks-nuoc-mia-nuoc-ep,2Go Drinks - Nước Mía & Nước Ép,2


### Restaurant

In [109]:
# remove duplicate restaurant
both_foody_df=both_foody_df[both_foody_df['url']!='chiaki-hiro-food-drink-nguyen-kim']
both_foody_df=both_foody_df[(both_foody_df['url']!='ca-vien-chien-176-nguyen-oanh') | (both_foody_df['view_count']!=474)]

In [129]:
final_shopee_df=both_shopee_df[[
        'restaurant_id', 'restaurant_url', 'restaurant_name', 'name_en',
       'restaurant_short_description', 'address_detail', 'address_district',
       'address_city', 'lat', 'lon', 'brand_id','delivery_avg_price', 'delivery_fees',
       'delivery_has_contract', 'delivery_id',
       'delivery_merchant_limit_distance', 'delivery_merchant_time',
       'delivery_payment_methods', 'delivery_prepare_duration',
       'delivery_service_by', 'delivery_setting_limit_distance',
       'delivery_shipping_fee_minimum', 'delivery_shipping_fee_rate',
       'limit_distance', 'asap_is_available', 'has_phone', 'is_city_alert',
       'is_display_cutlery', 'is_quality_merchant', 'contract_type',
       'position_verifie', 'foody_service_id', 'parent_category_id',
       'min_order_value', 'price_range_max_price', 'price_range_min_price',
       'total_like', 'total_order', 'rating_avg', 'rating_total_review','promotion_count'
]]

In [130]:
final_foody_df=both_foody_df[[
    'restaurant_id', 'full_address','area_address', 'status', 'min_price',
       'max_price', 'average_rating', 'position_rating', 'price_rating',
       'quality_rating', 'service_rating', 'space_rating', 'checked_in_count',
       'favourite_count', 'review_average_count', 'review_bad_count',
       'review_count', 'review_excellent_count', 'review_good_count',
       'view_count', 'wanted_count', 'picture_count'
]]

In [131]:
restaurant_df=final_shopee_df.merge(final_foody_df,on='restaurant_id',how='inner')

In [132]:
restaurant_df

,restaurant_id,restaurant_url,restaurant_name,name_en,restaurant_short_description,address_detail,address_district,address_city,lat,lon,brand_id,delivery_avg_price,delivery_fees,delivery_has_contract,delivery_id,delivery_merchant_limit_distance,delivery_merchant_time,delivery_payment_methods,delivery_prepare_duration,delivery_service_by,delivery_setting_limit_distance,delivery_shipping_fee_minimum,delivery_shipping_fee_rate,limit_distance,asap_is_available,has_phone,is_city_alert,is_display_cutlery,is_quality_merchant,contract_type,position_verifie,foody_service_id,parent_category_id,min_order_value,price_range_max_price,price_range_min_price,total_like,total_order,rating_avg,rating_total_review,promotion_count,full_address,area_address,status,min_price,max_price,average_rating,position_rating,price_rating,quality_rating,service_rating,space_rating,checked_in_count,favourite_count,review_average_count,review_bad_count,review_count,review_excellent_count,review_good_count,view_count,wanted_count,picture_count
0,786442,lien-hoa-ga-ac-oc-heo-tiem-thuoc-bac,Liên Hoa - Gà Ác & Óc Heo Tiềm Thuốc Bắc,NaN,NaN,"582 Quốc Lộ 13, P. Hiệp Bình Phước",Thủ Đức,TP. HCM,10.850111,106.719257,NaN,55000,"[{""note"": {""text"": {""message"": ""This fee appli...",False,62676,-1,12,"[6, 1, 4, 8]",12,ShopeeFood,20000,15000.0,1.0,20000,False,True,True,True,0,2,False,1,NaN,20000,60000,45000,0.0,343.0,4.8,10,1,"582 Quốc Lộ 13, P. Hiệp Bình Phước, Quận Thủ Đ...",Khu vực Ngã 4 Bình Phước,Đang hoạt động,45000.0,60000.0,NaN,--,--,--,--,--,0.0,1.0,NaN,NaN,0.0,NaN,NaN,230.0,0.0,0.0
1,14,ngoc-suong-ben-thuyen-nguyen-van-troi,NS Bến Thuyền,Ngoc Suong Restaurant,NaN,"11 Nguyễn Văn Trỗi, P. 12",Phú Nhuận,TP. HCM,10.791926,106.681183,NaN,71839,"[{""note"": {""text"": {""message"": ""This fee appli...",False,33938,-1,12,"[6, 1, 4, 8]",12,ShopeeFood,20000,15000.0,1.0,20000,False,True,False,True,0,2,True,1,2546.0,20000,1100000,200000,6.0,597.0,4.8,100,1,"11 Nguyễn Văn Trỗi, P. 12, Quận Phú Nhuận, TP....",Khu vực Nguyễn Trọng Tuyển,Đang hoạt động,200000.0,1100000.0,7.3,7.4,6.6,7.8,7.2,7.6,51.0,188.0,23.0,6.0,115.0,8.0,78.0,79232.0,129.0,830.0
2,50,passio-coffee-15f-nguyen-thi-minh-khai,Passio Coffee - 15F Nguyễn Thị Minh Khai,Passio Coffee,NaN,"15F Nguyễn Thị Minh Khai, P. Bến Nghé",Quận 1,TP. HCM,10.786102,106.701135,282.0,36467,[],True,8077,-1,12,"[6, 1, 4, 8]",12,ShopeeFood,20000,15000.0,1.0,20000,False,True,False,True,0,1,True,1,2448.0,20000,55000,30000,1071.0,39627.0,4.9,1000,17,"15F Nguyễn Thị Minh Khai, P. Bến Nghé, Quận 1,...",Khu vực Đài Truyền Hình,Đang hoạt động,30000.0,55000.0,7.1,7.6,7.0,7.4,7.1,6.7,75.0,76.0,29.0,4.0,130.0,3.0,94.0,64530.0,35.0,332.0
3,786494,bun-mi-vang-hu-tieu-mi-thap-cam,Bún Mì Vàng & Hủ Tiếu Mì Thập Cẩm,NaN,NaN,"89 Trần Quý, P. 4",Quận 11,TP. HCM,10.758493,106.653401,NaN,50800,"[{""note"": {""text"": {""message"": ""This fee appli...",False,69508,-1,12,"[6, 1, 4, 8]",12,ShopeeFood,20000,15000.0,1.0,20000,False,True,True,True,0,2,False,1,2387.0,20000,46000,38000,6.0,824.0,4.8,10,1,"89 Trần Quý, P. 4, Quận 11, TP. HCM",Khu vực Chợ Thiếc,Đang hoạt động,38000.0,46000.0,7.0,7.5,6.0,8.0,6.0,7.5,0.0,7.0,1.0,0.0,2.0,0.0,1.0,443.0,2.0,12.0
4,73,hai-san-be-vang,Hải Sản Bê Vàng,Bê Vàng,NaN,"261 - 263 An Dương Vương, P. 3",Quận 5,TP. HCM,10.759281,106.678682,NaN,85600,"[{""note"": {""text"": {""message"": ""This fee appli...",False,8275,-1,12,"[6, 1, 4, 8]",12,ShopeeFood,20000,15000.0,1.0,20000,False,True,True,True,0,2,True,1,2546.0,20000,550000,300000,9.0,1460.0,4.9,50,1,"261 - 263 An Dương Vương, P. 3, Quận 5, TP. HCM",Khu vực Nowzone - Nguyễn Văn Cừ,Đang hoạt động,300000.0,550000.0,7.6,7.5,6.7,8.3,7.5,7.9,22.0,112.0,5.0,3.0,59.0,5.0,46.0,41161.0,120.0,569.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61576,1083136,vu-heo-nuong-family-bbq,

In [133]:
restaurant_df['brand_id']=restaurant_df['brand_id'].fillna(-1).astype(int)
restaurant_df['delivery_has_contract']=restaurant_df['delivery_has_contract'].astype(int)

restaurant_df['asap_is_available']=restaurant_df['asap_is_available'].astype(int)
restaurant_df['has_phone']=restaurant_df['has_phone'].astype(int)
restaurant_df['is_city_alert']=restaurant_df['is_city_alert'].astype(int)
restaurant_df['is_display_cutlery']=restaurant_df['is_display_cutlery'].astype(int)
restaurant_df['position_verifie']=restaurant_df['position_verifie'].astype(int)

restaurant_df['parent_category_id']=restaurant_df['parent_category_id'].fillna(-1).astype(int)

restaurant_df['position_rating']=restaurant_df['position_rating'].replace('--','nan').astype(float)
restaurant_df['price_rating']=restaurant_df['price_rating'].replace('--','nan').astype(float)
restaurant_df['quality_rating']=restaurant_df['quality_rating'].replace('--','nan').astype(float)
restaurant_df['service_rating']=restaurant_df['service_rating'].replace('--','nan').astype(float)
restaurant_df['space_rating']=restaurant_df['space_rating'].replace('--','nan').astype(float)

restaurant_df['checked_in_count']=restaurant_df['checked_in_count'].astype(int)
restaurant_df['favourite_count']=restaurant_df['favourite_count'].astype(int)

restaurant_df['price_range_min_price']=restaurant_df['price_range_min_price'].replace(0,np.nan)
restaurant_df['price_range_max_price']=restaurant_df['price_range_max_price'].replace(0,np.nan)

### Dishes

In [116]:
root_path="drive/MyDrive/HCMUS/2022 - 2023/Data/1 - raw data/summary/"
db='Main 2022 10 23.db'
_root = Path(root_path)
db_path=_root.joinpath(db)
#----------------------------------------
try: 
    conn = lite.connect(db_path)
    print(f"sqlite3 {lite.version} has connected to database successfully")
except lite.Error as e:
    print("Error: ", e)

sqlite3 2.6.0 has connected to database successfully


In [138]:
#----------------------------------------
with conn:
  cur = conn.cursor()
  cur.execute("SELECT restaurant_id FROM RESTAURANT")
  result=cur.fetchall()
#---------------------------------------
list_both_restaurant_id=[x[0] for x in result]


In [139]:
!ls drive/MyDrive/HCMUS/'2022 - 2023'/Data/'1 - raw data'/shopee/'shopee dish'/dishes

 dishes_8k.xlsx		  'dishes_Phú Nhuận.xlsx'  'dishes_Quận 6.xlsx'
'dishes_Bình Chánh.xlsx'  'dishes_Quận 10.xlsx'    'dishes_Quận 7.xlsx'
'dishes_Bình Tân.xlsx'	  'dishes_Quận 11.xlsx'    'dishes_Quận 8.xlsx'
'dishes_Bình Thạnh.xlsx'  'dishes_Quận 12.xlsx'    'dishes_Quận 9.xlsx'
'dishes_Cần Giờ.xlsx'	  'dishes_Quận 1.xlsx'	   'dishes_Tân Bình.xlsx'
'dishes_Củ Chi.xlsx'	  'dishes_Quận 2.xlsx'	   'dishes_Tân Phú.xlsx'
'dishes_Gò Vấp.xlsx'	  'dishes_Quận 3.xlsx'	   'dishes_Thủ Đức.xlsx'
'dishes_Hóc Môn.xlsx'	  'dishes_Quận 4.xlsx'
'dishes_Nhà Bè.xlsx'	  'dishes_Quận 5.xlsx'


In [140]:
list_dish_file=os.listdir('drive/MyDrive/HCMUS/2022 - 2023/Data/1 - raw data/shopee/shopee dish/dishes')

In [147]:
for file in list_dish_file:
  temp_df=pd.read_excel(os.path.join('drive/MyDrive/HCMUS/2022 - 2023/Data/1 - raw data/shopee/shopee dish/dishes',file))
  temp_df[temp_df['dish_restaurant_id'].isin(list_both_restaurant_id)].drop(columns={'Unnamed: 0'}).to_sql('DIM_SHOPEE_DISHES',conn, if_exists='append',index=False)

In [157]:
dish_df=pd.read_sql('SELECT * FROM DIM_SHOPEE_DISHES',conn)

In [158]:
dish_df['dish_price']=dish_df['dish_price'].astype(int)

In [160]:
                                                dish_df.sort_values(by='dish_price',ascending=False).head(20)

,catalog_id,dish_total_order,catalog_name,catalog_rank,catalog_partner_catalog_id,catalog_description,dish_restaurant_id,dish_id,dish_name,dish_partner_dish_id,dish_listing_status,dish_description,dish_total_like,dish_rank,dish_picture_label,dish_is_hidden,dish_price,dish_is_group_discount_item,dishes_property_info
1741385,5176523,0,test,0,None,None,1150623,118678652,test,None,1,None,0,0,None,0,1110000000000000,1,"{""has_alcohol"": false}"
670627,620990,0,MENU,0,None,None,920258,32848803,"Combo bánh xèo, khọt 4 phần/ tặng 1 chai nước sâm",None,1,Bánh xèo 1 phần 30000 bánh khọt 1 phần 35000,0,0,None,0,120000140000,1,"{""has_alcohol"": false}"
22654,2899113,0,Bánh mì đen healthy,0,None,None,1000046737,70231583,Feedback khách hàng,None,1,❤️ Cảm ơn Bạn đã tin dùng sản phẩm.. \n==> Ti...,0,0,None,0,999999999,1,"{""has_alcohol"": false}"
1526624,3354529,0,Thông Báo Của Quán,0,None,None,1000048115,64200100,Mọi SAI Hoặc THIẾU SÓT khách hãy Gọi/Nhắn 0908...,None,1,"Khách ơi, hãy để quán sửa và đền bù cho bạn nế...",0,0,None,0,908012012,1,"{""has_alcohol"": false}"
490843,2462708,2,Menu Món Chế Biến Sẵn,0,None,None,1117695,32170826,Râu mực nướng,None,1,Râu mực nướng 1 phần 60k tặng kèm rau ăn kèm,2,0,None,0,70000000,1,"{""has_alcohol"": false}"
182504,1949328,8,Menu,0,None,None,1092525,18185377,Chả giò chay - 20 cuốn,None,1,None,0,0,None,0,59000966,1,"{""has_alcohol"": false}"
438220,2215839,0,Món Chính,0,None,None,1000033737,25226722,Bún bò gân,None,1,None,0,0,None,0,35000000,1,"{""has_alcohol"": false}"
1179812,2216627,0,Menu,0,None,None,1000033769,103842829,Milo đá dầm,None,1,Milo \nBánh plan\nTrân châu đường đen,0,0,None,0,27000000,1,"{""has_alcohol"": false}"
980226,2751760,0,Trà,2,None,None,1000045527,40762091,Sữa tươi trân châu đường đen 12oz,None,1,None,0,0,None,0,25000000,1,"{""has_alcohol"": false}"
1167738,2266637,0,Menu,0,None,None,1112008,26446756,Yến tổ Huyết (Đảo sạch) - 100gr,None,1,None,0,0,None,0,21000000,1,"{""has_alcohol"": false}"


In [ ]:
conn.close()

# Write to sqlite

## Create table

### Shopee

In [119]:
#active_hour shopee
with conn:
    cur = conn.cursor()
    cur.execute("CREATE TABLE IF NOT EXISTS DIM_ACTIVE_HOURS (\
        restaurant_id INTEGER\
        ,week_day INTEGER\
        ,active_hour INTEGER\
        ,UNIQUE(restaurant_id,week_day,active_hour)\
      )")

#shopee category
with conn:
    cur=conn.cursor()
    cur.execute("CREATE TABLE IF NOT EXISTS DIM_SHOPEE_CATEGORY(\
    restaurant_id INTEGER\
    ,category text\
    ,UNIQUE(restaurant_id,category)\
    )")


#shopee cuisine
with conn:
    cur=conn.cursor()
    cur.execute("CREATE TABLE IF NOT EXISTS DIM_SHOPEE_CUISINE(\
    restaurant_id INTEGER\
    ,cuisine text\
    ,UNIQUE(restaurant_id,cuisine)\
    )")


#shopee root-category
with conn:
    cur=conn.cursor()
    cur.execute("CREATE TABLE IF NOT EXISTS DIM_SHOPEE_ROOT_CATEGORY(\
    restaurant_id INTEGER\
    ,root_category text\
    ,UNIQUE(restaurant_id,root_category)\
    )")
#shopee brand
with conn:
    cur=conn.cursor()
    cur.execute("CREATE TABLE IF NOT EXISTS DIM_SHOPEE_BRAND(\
    brand_id INTEGER\
    ,brand_url text\
    ,brand_name text\
    ,restaurant_count INTEGER\
    ,UNIQUE(brand_id,brand_url)\
    )")
#shopee dishes
with conn:
    cur=conn.cursor()
    cur.execute("CREATE TABLE IF NOT EXISTS DIM_SHOPEE_DISHES(\
        catalog_id                       INTEGER\
        ,dish_total_order                 INTEGER\
        ,catalog_name                    TEXT\
        ,catalog_rank                     INTEGER\
        ,catalog_partner_catalog_id      TEXT\
        ,catalog_description            TEXT\
        ,dish_restaurant_id               INTEGER\
        ,dish_id                          INTEGER\
        ,dish_name                       TEXT\
        ,dish_partner_dish_id            TEXT\
        ,dish_listing_status               INTEGER\
        ,dish_description                TEXT\
        ,dish_total_like                  INTEGER\
        ,dish_rank                        INTEGER\
        ,dish_picture_label             TEXT\
        ,dish_is_hidden                    INTEGER\
        ,dish_price                       FLOAT\
        ,dish_is_group_discount_item       INTEGER\
        ,dishes_property_info            TEXT\
    )")


In [120]:
active_hour_df.to_sql('DIM_ACTIVE_HOURS',conn, if_exists='replace',index=False)
category_shopee_df.to_sql('DIM_SHOPEE_CATEGORY',conn, if_exists='replace',index=False)
cuisine_shopee_df.to_sql('DIM_SHOPEE_CUISINE',conn, if_exists='replace',index=False)
root_category_shopee_df.to_sql('DIM_SHOPEE_ROOT_CATEGORY',conn, if_exists='replace',index=False)


In [121]:
brand_shopee_df.to_sql('DIM_SHOPEE_BRAND',conn, if_exists='replace',index=False)

### Foody

In [124]:
#foody cuisine
with conn:
    cur=conn.cursor()
    cur.execute("CREATE TABLE IF NOT EXISTS DIM_FOODY_CUISINE(\
    restaurant_id INTEGER\
    ,cuisine text\
    ,UNIQUE(restaurant_id,cuisine)\
    )")

#foody category
with conn:
    cur=conn.cursor()
    cur.execute("CREATE TABLE IF NOT EXISTS DIM_FOODY_CATEGORY(\
    restaurant_id INTEGER\
    ,category text\
    ,UNIQUE(restaurant_id,category)\
    )")

#foody audience
with conn:
    cur=conn.cursor()
    cur.execute("CREATE TABLE IF NOT EXISTS DIM_FOODY_AUDIENCE(\
    restaurant_id INTEGER\
    ,audience text\
    ,UNIQUE(restaurant_id,audience)\
    )")

In [125]:
cuisine_foody_df.to_sql('DIM_FOODY_CUISINE',conn,if_exists='replace',index=False)
category_foody_df.to_sql('DIM_FOODY_CATEGORY',conn,if_exists='replace',index=False)
audience_foody_df.to_sql('DIM_FOODY_AUDIENCE',conn,if_exists='replace',index=False)

### Restaurant

In [135]:
with conn:
    cur=conn.cursor()
    cur.execute("CREATE TABLE IF NOT EXISTS RESTAURANT(\
restaurant_id INTEGER,  restaurant_url TEXT,  restaurant_name TEXT,  name_en TEXT,  restaurant_short_description TEXT,  address_detail TEXT\
,  address_district TEXT,  address_city TEXT,  lat FLOAT,  lon FLOAT,  brand_id INTEGER,  delivery_avg_price FLOAT,  delivery_fees TEXT,  delivery_has_contract INTEGER\
,  delivery_id INTEGER,  delivery_merchant_limit_distance INTEGER,  delivery_merchant_time INTEGER,  delivery_payment_methods TEXT,  delivery_prepare_duration INTEGER\
,  delivery_service_by TEXT,  delivery_setting_limit_distance INTEGER,  delivery_shipping_fee_minimum FLOAT,  delivery_shipping_fee_rate FLOAT,  limit_distance INTEGER\
,  asap_is_available INTEGER,  has_phone INTEGER,  is_city_alert INTEGER,  is_display_cutlery INTEGER,  is_quality_merchant INTEGER,  contract_type INTEGER\
,  position_verifie INTEGER,  foody_service_id INTEGER,  parent_category_id INTEGER,  min_order_value FLOAT,  price_range_max_price FLOAT,  price_range_min_price FLOAT\
,  total_like INTEGER,  total_order INTEGER,  rating_avg FLOAT,  rating_total_review INTEGER,  promotion_count INTEGER,  full_address TEXT,area_address TEXT,  status TEXT,  min_price FLOAT\
,  max_price FLOAT,  average_rating FLOAT,  position_rating FLOAT,  price_rating FLOAT,  quality_rating FLOAT,  service_rating FLOAT,  space_rating FLOAT,  checked_in_count INTEGER\
,  favourite_count INTEGER,  review_average_count INTEGER,  review_bad_count INTEGER,  review_count INTEGER,  review_excellent_count INTEGER,  review_good_count INTEGER\
,  view_count INTEGER,  wanted_count INTEGER,  picture_count INTEGER,\
UNIQUE(restaurant_id))")

In [137]:
restaurant_df.to_sql('RESTAURANT',conn,if_exists='replace',index=False)

In [ ]:
conn.close()